RANDOM FOREST WITH SMOTEENN + PCA
RECALL IS GETTING HIGH BUT ACCURACY AND PRECISION NOT
I HAVE NOTE RESULT RF+ADASYN+PCA,RF+SMOTE+PCA

In [19]:
import pandas as pd
from collections import Counter
df = pd.read_csv("/home/roopal/diabetes_data_preprocessed_big.csv")

In [20]:
df['readmitted'] = df['readmitted'].apply(lambda x: 0 if x == 2 else x)

In [21]:
df1 = df[['race','gender','age','admission_type_id','discharge_disposition_id','admission_source_id','time_in_hospital','num_lab_procedures','num_procedures','num_medications','number_outpatient','number_emergency','number_inpatient','diag_1','diag_2','diag_3','number_diagnoses','max_glu_serum','A1Cresult','insulin','change','diabetesMed']]
df1 = pd.get_dummies(df1, columns=['race'], drop_first = True)
y = df['readmitted']
print('Original dataset shape {}'.format(Counter(y)))
import numpy as np
df['number_outpatient'] = np.log1p(df['number_outpatient'])
df['number_inpatient'] = np.log1p(df['number_inpatient'])
df['number_emergency'] = np.log1p(df['number_emergency'])


Original dataset shape Counter({0: 88764, 1: 11357})


In [22]:
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTEENN
smt = SMOTEENN(random_state=20,sampling_strategy='all',ratio=None)
X_train, X_test, Y_train, Y_test = train_test_split(df1,y, test_size=0.20, random_state=0)
x_train, y_train = smt.fit_sample(X_train, Y_train)
print('Now dataset shape {}'.format(Counter(y_train)))

Now dataset shape Counter({1: 69476, 0: 37396})


In [6]:
from sklearn.preprocessing import StandardScaler
#X = x_train.values
X_std = StandardScaler().fit_transform(x_train)

In [7]:
mean_vec = np.mean(X_std, axis=0)
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
# Create a list of (eigenvalue, eigenvector) tuples
eig_pairs = [ (np.abs(eig_vals[i]),eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the eigenvalue, eigenvector pair from high to low
eig_pairs.sort(key = lambda x: x[0], reverse= True)

# Calculation of Explained Variance from the eigenvalues
tot = sum(eig_vals)
var_exp = [(i/tot)*100 for i in sorted(eig_vals, reverse=True)] # Individual explained variance
cum_var_exp = np.cumsum(var_exp) 

In [8]:
[ n for n,i in enumerate(cum_var_exp) if i>90 ][0]

18

In [9]:
from sklearn.decomposition import PCA
pca = PCA(n_components=19)
pca.fit(X_std)
X_228d = pca.transform(X_std)


In [10]:
from sklearn.ensemble import RandomForestClassifier as RF


In [11]:
y = X_test.values
y_std = StandardScaler().fit_transform(y)
from sklearn.decomposition import PCA

y_228d = pca.transform(y_std)

In [15]:
from sklearn.metrics import classification_report 
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score,f1_score
import pickle
acc = []
precision = []
recall = []
auc = []
f1 = []
f= open("tr.txt","a+")
f.write('accuracy'+","+'Precision'+","+'Recall'+","+'AUC')
f.write("\n")
for i in [50,100,200,250,300]:
    for j in [2,3,4,5,7,9,11,15,18,20,23,25]:
        clf = RF(n_estimators = i,max_depth = j)
        clf = clf.fit(X_228d,y_train)
        pred1 = clf.predict(X_228d)
        #print("Accuracy is {0:.2f}".format(accuracy_score(y_train, pred1)))
        acc = accuracy_score(y_train, pred1)
        #print("Precision is {0:.2f}".format(precision_score(y_train, pred1)))
        precision = precision_score(y_train, pred1)
        #print("Recall is {0:.2f}".format(recall_score(y_train, pred1)))
        recall = recall_score(y_train, pred1)
        #print("AUC is {0:.2f}".format(roc_auc_score(y_train,pred1)))
        auc = roc_auc_score(y_train,pred1)
        #print("f1 is {0:.2f}".format(f1_score(y_train,pred1)))
        f1 = f1_score(y_train,pred1)
        print(classification_report(y_train,pred1))
        file='/home/roopal/rf_model/'+str(i)+str(j)+'rf.pkl'
        
        pickle.dump(clf, open(file, 'wb'))
        
        f.write(str(acc)+","+str(precision)+","+str(recall)+","+str(auc)+","+str(f1))
        f.write("\n")
        
#f.write("\n".join(precision))
#f.write("\n".join(recall))
#f.write("\n".join(f1))
    
f.close()

              precision    recall  f1-score   support

           0       0.89      0.00      0.01     37396
           1       0.65      1.00      0.79     69476

   micro avg       0.65      0.65      0.65    106872
   macro avg       0.77      0.50      0.40    106872
weighted avg       0.73      0.65      0.52    106872

              precision    recall  f1-score   support

           0       0.85      0.04      0.08     37396
           1       0.66      1.00      0.79     69476

   micro avg       0.66      0.66      0.66    106872
   macro avg       0.75      0.52      0.44    106872
weighted avg       0.72      0.66      0.54    106872

              precision    recall  f1-score   support

           0       0.83      0.09      0.17     37396
           1       0.67      0.99      0.80     69476

   micro avg       0.68      0.68      0.68    106872
   macro avg       0.75      0.54      0.48    106872
weighted avg       0.73      0.68      0.58    106872

              preci

              precision    recall  f1-score   support

           0       0.84      0.09      0.17     37396
           1       0.67      0.99      0.80     69476

   micro avg       0.68      0.68      0.68    106872
   macro avg       0.75      0.54      0.48    106872
weighted avg       0.73      0.68      0.58    106872

              precision    recall  f1-score   support

           0       0.82      0.14      0.24     37396
           1       0.68      0.98      0.80     69476

   micro avg       0.69      0.69      0.69    106872
   macro avg       0.75      0.56      0.52    106872
weighted avg       0.73      0.69      0.61    106872

              precision    recall  f1-score   support

           0       0.83      0.24      0.38     37396
           1       0.71      0.97      0.82     69476

   micro avg       0.72      0.72      0.72    106872
   macro avg       0.77      0.61      0.60    106872
weighted avg       0.75      0.72      0.66    106872

              preci

              precision    recall  f1-score   support

           0       0.83      0.24      0.38     37396
           1       0.70      0.97      0.82     69476

   micro avg       0.72      0.72      0.72    106872
   macro avg       0.77      0.61      0.60    106872
weighted avg       0.75      0.72      0.66    106872

              precision    recall  f1-score   support

           0       0.89      0.34      0.49     37396
           1       0.73      0.98      0.84     69476

   micro avg       0.75      0.75      0.75    106872
   macro avg       0.81      0.66      0.66    106872
weighted avg       0.79      0.75      0.72    106872

              precision    recall  f1-score   support

           0       0.95      0.45      0.61     37396
           1       0.77      0.99      0.87     69476

   micro avg       0.80      0.80      0.80    106872
   macro avg       0.86      0.72      0.74    106872
weighted avg       0.83      0.80      0.78    106872

              preci

In [13]:
Y_train.shape

(80096,)

In [17]:
pred

array([1, 1, 1, ..., 1, 0, 1])

              precision    recall  f1-score   support

           0       0.97      0.00      0.00     37396
           1       0.65      1.00      0.79     69476

   micro avg       0.65      0.65      0.65    106872
   macro avg       0.81      0.50      0.39    106872
weighted avg       0.76      0.65      0.51    106872

              precision    recall  f1-score   support

           0       0.86      0.04      0.07     37396
           1       0.66      1.00      0.79     69476

   micro avg       0.66      0.66      0.66    106872
   macro avg       0.76      0.52      0.43    106872
weighted avg       0.73      0.66      0.54    106872

              precision    recall  f1-score   support

           0       0.97      0.00      0.00     37396
           1       0.65      1.00      0.79     69476

   micro avg       0.65      0.65      0.65    106872
   macro avg       0.81      0.50      0.40    106872
weighted avg       0.76      0.65      0.51    106872

              preci

In [12]:
from sklearn.metrics import classification_report 
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score,f1_score
f= open("testre.txt","a+")
import pickle
f.write('accuracy'+","+'Precision'+","+'Recall'+","+'AUC')
f.write("\n")
acc = []
precision = []
recall = []
auc = []
f1 = []

for i in [50,100,200,250,300]:
    for j in [2,3,4,5,7,9,11,15,18,20,23,25]:
        file='/home/roopal/rf_model/'+str(i)+str(j)+'rf.pkl'
        load_clf = pickle.load(open(file, 'rb'))
        
        load_clf.fit(X_228d,y_train)
        pred1 = load_clf.predict(y_228d)
        #print("Accuracy is {0:.2f}".format(accuracy_score(y_train, pred1)))
        acc = accuracy_score(Y_test, pred1)
        #print("Precision is {0:.2f}".format(precision_score(y_train, pred1)))
        precision = precision_score(Y_test, pred1)
        #print("Recall is {0:.2f}".format(recall_score(y_train, pred1)))
        recall = recall_score(Y_test, pred1)
        #print("AUC is {0:.2f}".format(roc_auc_score(y_train,pred1)))
        auc = roc_auc_score(Y_test,pred1)
        #print("f1 is {0:.2f}".format(f1_score(y_train,pred1)))
        f1 = f1_score(Y_test,pred1)
        print(classification_report(Y_test,pred1))
        f.write(str(acc)+","+str(precision)+","+str(recall)+","+str(auc)+","+str(f1))
        f.write("\n")
        
#f.write("\n".join(precision))
#f.write("\n".join(recall))
#f.write("\n".join(f1))
    
f.close()

              precision    recall  f1-score   support

           0       1.00      0.00      0.00     17784
           1       0.11      1.00      0.20      2241

   micro avg       0.11      0.11      0.11     20025
   macro avg       0.56      0.50      0.10     20025
weighted avg       0.90      0.11      0.02     20025

              precision    recall  f1-score   support

           0       1.00      0.01      0.01     17784
           1       0.11      1.00      0.20      2241

   micro avg       0.12      0.12      0.12     20025
   macro avg       0.56      0.50      0.11     20025
weighted avg       0.90      0.12      0.03     20025

              precision    recall  f1-score   support

           0       0.95      0.03      0.05     17784
           1       0.11      0.99      0.20      2241

   micro avg       0.13      0.13      0.13     20025
   macro avg       0.53      0.51      0.13     20025
weighted avg       0.86      0.13      0.07     20025

              preci

              precision    recall  f1-score   support

           0       0.95      0.03      0.06     17784
           1       0.11      0.99      0.20      2241

   micro avg       0.14      0.14      0.14     20025
   macro avg       0.53      0.51      0.13     20025
weighted avg       0.86      0.14      0.08     20025

              precision    recall  f1-score   support

           0       0.95      0.05      0.10     17784
           1       0.12      0.98      0.21      2241

   micro avg       0.16      0.16      0.16     20025
   macro avg       0.53      0.52      0.15     20025
weighted avg       0.86      0.16      0.11     20025

              precision    recall  f1-score   support

           0       0.94      0.09      0.16     17784
           1       0.12      0.96      0.21      2241

   micro avg       0.18      0.18      0.18     20025
   macro avg       0.53      0.52      0.18     20025
weighted avg       0.85      0.18      0.17     20025

              preci

/home/roopal/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00     17784
           1       0.11      1.00      0.20      2241

   micro avg       0.11      0.11      0.11     20025
   macro avg       0.06      0.50      0.10     20025
weighted avg       0.01      0.11      0.02     20025

              precision    recall  f1-score   support

           0       0.98      0.01      0.02     17784
           1       0.11      1.00      0.20      2241

   micro avg       0.12      0.12      0.12     20025
   macro avg       0.55      0.50      0.11     20025
weighted avg       0.88      0.12      0.04     20025

              precision    recall  f1-score   support

           0       0.95      0.03      0.06     17784
           1       0.11      0.99      0.20      2241

   micro avg       0.14      0.14      0.14     20025
   macro avg       0.53      0.51      0.13     20025
weighted avg       0.86      0.14      0.07     20025

              preci

In [14]:
'''from sklearn.metrics import classification_report 
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score,f1_score
f= open("d.txt","a+")
acc = []
precision = []
recall = []
auc = []
f1 = []
f.write('accuracy'+","+'Precision'+","+'Recall'+","+'AUC')
for i in [50]:
    for j in [2,3,4]:
        clf = RF(n_estimators = i,max_depth = j)
        clf = clf.fit(X_228d,y_train)
        pred1 = clf.predict(X_228d)
        #print("Accuracy is {0:.2f}".format(accuracy_score(y_train, pred1)))
        acc = accuracy_score(y_train, pred1)
        #print("Precision is {0:.2f}".format(precision_score(y_train, pred1)))
        precision = precision_score(y_train, pred1)
        #print("Recall is {0:.2f}".format(recall_score(y_train, pred1)))
        recall = recall_score(y_train, pred1)
        #print("AUC is {0:.2f}".format(roc_auc_score(y_train,pred1)))
        auc = roc_auc_score(y_train,pred1)
        #print("f1 is {0:.2f}".format(f1_score(y_train,pred1)))
        f1 = f1_score(y_train,pred1)
        print(classification_report(y_train,pred1))
        f.write(str(acc)+","+str(precision)+","+str(recall)+","+str(auc))
        f.write("\n")
        
#f.write("\n".join(precision))
#f.write("\n".join(recall))
#f.write("\n".join(f1))
    
f.close()'''

'from sklearn.metrics import classification_report \nfrom sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score,f1_score\nf= open("d.txt","a+")\nacc = []\nprecision = []\nrecall = []\nauc = []\nf1 = []\nf.write(\'accuracy\'+","+\'Precision\'+","+\'Recall\'+","+\'AUC\')\nfor i in [50]:\n    for j in [2,3,4]:\n        clf = RF(n_estimators = i,max_depth = j)\n        clf = clf.fit(X_228d,y_train)\n        pred1 = clf.predict(X_228d)\n        #print("Accuracy is {0:.2f}".format(accuracy_score(y_train, pred1)))\n        acc = accuracy_score(y_train, pred1)\n        #print("Precision is {0:.2f}".format(precision_score(y_train, pred1)))\n        precision = precision_score(y_train, pred1)\n        #print("Recall is {0:.2f}".format(recall_score(y_train, pred1)))\n        recall = recall_score(y_train, pred1)\n        #print("AUC is {0:.2f}".format(roc_auc_score(y_train,pred1)))\n        auc = roc_auc_score(y_train,pred1)\n        #print("f1 is {0:.2f}".format(f1_

In [ ]:
import pandas as pd
from collections import Counter
df = pd.read_csv("/home/roopal/diabetes_data_preprocessed_big.csv")
df['readmitted'] = df['readmitted'].apply(lambda x: 0 if x == 2 else x)
df1 = df[['race','gender','age','admission_type_id','discharge_disposition_id','admission_source_id','time_in_hospital','num_lab_procedures','num_procedures','num_medications','number_outpatient','number_emergency','number_inpatient','diag_1','diag_2','diag_3','number_diagnoses','max_glu_serum','A1Cresult','insulin','change','diabetesMed']]
df1 = pd.get_dummies(df1, columns=['race'], drop_first = True)
y = df['readmitted']
print('Original dataset shape {}'.format(Counter(y)))
import numpy as np
df['number_outpatient'] = np.log1p(df['number_outpatient'])
df['number_inpatient'] = np.log1p(df['number_inpatient'])
df['number_emergency'] = np.log1p(df['number_emergency'])
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
smt = SMOTE(random_state=20,sampling_strategy='all',ratio=None)
X_train, X_test, Y_train, Y_test = train_test_split(df1,y, test_size=0.20, random_state=0)
x_train, y_train = smt.fit_sample(X_train, Y_train)
print('Now dataset shape {}'.format(Counter(y_train)))
from sklearn.preprocessing import StandardScaler
#X = x_train.values
X_std = StandardScaler().fit_transform(x_train)
mean_vec = np.mean(X_std, axis=0)
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
# Create a list of (eigenvalue, eigenvector) tuples
eig_pairs = [ (np.abs(eig_vals[i]),eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the eigenvalue, eigenvector pair from high to low
eig_pairs.sort(key = lambda x: x[0], reverse= True)

# Calculation of Explained Variance from the eigenvalues
tot = sum(eig_vals)
var_exp = [(i/tot)*100 for i in sorted(eig_vals, reverse=True)] # Individual explained variance
cum_var_exp = np.cumsum(var_exp) 
[ n for n,i in enumerate(cum_var_exp) if i>90 ][0]
from sklearn.decomposition import PCA
pca = PCA(n_components=19)
pca.fit(X_std)
X_228d = pca.transform(X_std)
from sklearn.ensemble import RandomForestClassifier as RF
y = X_test.values
y_std = StandardScaler().fit_transform(y)
from sklearn.decomposition import PCA

y_228d = pca.transform(y_std)
from sklearn.metrics import classification_report 
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score,f1_score
import pickle
acc = []
precision = []
recall = []
auc = []
f1 = []
f= open("tr.txt","a+")
f.write('accuracy'+","+'Precision'+","+'Recall'+","+'AUC')
f.write("\n")
for i in [50,100,200,250,300]:
    for j in [2,3,4,5,7,9,11,15,18,20,23,25]:
        clf = RF(n_estimators = i,max_depth = j)
        clf = clf.fit(X_228d,y_train)
        pred1 = clf.predict(X_228d)
        #print("Accuracy is {0:.2f}".format(accuracy_score(y_train, pred1)))
        acc = accuracy_score(y_train, pred1)
        #print("Precision is {0:.2f}".format(precision_score(y_train, pred1)))
        precision = precision_score(y_train, pred1)
        #print("Recall is {0:.2f}".format(recall_score(y_train, pred1)))
        recall = recall_score(y_train, pred1)
        #print("AUC is {0:.2f}".format(roc_auc_score(y_train,pred1)))
        auc = roc_auc_score(y_train,pred1)
        #print("f1 is {0:.2f}".format(f1_score(y_train,pred1)))
        f1 = f1_score(y_train,pred1)
        print(classification_report(y_train,pred1))
        file='/home/roopal/rf_model/'+str(i)+str(j)+'rf.pkl'
        
        pickle.dump(clf, open(file, 'wb'))
        
        f.write(str(acc)+","+str(precision)+","+str(recall)+","+str(auc)+","+str(f1))
        f.write("\n")
        
#f.write("\n".join(precision))
#f.write("\n".join(recall))
#f.write("\n".join(f1))
    
f.close()

from sklearn.metrics import classification_report 
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score,f1_score
f= open("testre.txt","a+")
import pickle
f.write('accuracy'+","+'Precision'+","+'Recall'+","+'AUC')
f.write("\n")
acc = []
precision = []
recall = []
auc = []
f1 = []

for i in [50,100,200,250,300]:
    for j in [2,3,4,5,7,9,11,15,18,20,23,25]:
        file='/home/roopal/rf_model/'+str(i)+str(j)+'rf.pkl'
        load_clf = pickle.load(open(file, 'rb'))
        
        load_clf.fit(X_228d,y_train)
        pred1 = load_clf.predict(y_228d)
        #print("Accuracy is {0:.2f}".format(accuracy_score(y_train, pred1)))
        acc = accuracy_score(Y_test, pred1)
        #print("Precision is {0:.2f}".format(precision_score(y_train, pred1)))
        precision = precision_score(Y_test, pred1)
        #print("Recall is {0:.2f}".format(recall_score(y_train, pred1)))
        recall = recall_score(Y_test, pred1)
        #print("AUC is {0:.2f}".format(roc_auc_score(y_train,pred1)))
        auc = roc_auc_score(Y_test,pred1)
        #print("f1 is {0:.2f}".format(f1_score(y_train,pred1)))
        f1 = f1_score(Y_test,pred1)
        print(classification_report(Y_test,pred1))
        f.write(str(acc)+","+str(precision)+","+str(recall)+","+str(auc)+","+str(f1))
        f.write("\n")
        
#f.write("\n".join(precision))
#f.write("\n".join(recall))
#f.write("\n".join(f1))
    
f.close()

Original dataset shape Counter({0: 88764, 1: 11357})
Now dataset shape Counter({0: 70980, 1: 70980})
              precision    recall  f1-score   support

           0       0.61      0.61      0.61     70980
           1       0.61      0.61      0.61     70980

   micro avg       0.61      0.61      0.61    141960
   macro avg       0.61      0.61      0.61    141960
weighted avg       0.61      0.61      0.61    141960

              precision    recall  f1-score   support

           0       0.62      0.60      0.61     70980
           1       0.61      0.64      0.63     70980

   micro avg       0.62      0.62      0.62    141960
   macro avg       0.62      0.62      0.62    141960
weighted avg       0.62      0.62      0.62    141960

              precision    recall  f1-score   support

           0       0.63      0.62      0.62     70980
           1       0.63      0.63      0.63     70980

   micro avg       0.63      0.63      0.63    141960
   macro avg       0.63    

              precision    recall  f1-score   support

           0       0.62      0.61      0.61     70980
           1       0.61      0.62      0.62     70980

   micro avg       0.61      0.61      0.61    141960
   macro avg       0.62      0.61      0.61    141960
weighted avg       0.62      0.61      0.61    141960

              precision    recall  f1-score   support

           0       0.63      0.62      0.63     70980
           1       0.63      0.65      0.64     70980

   micro avg       0.63      0.63      0.63    141960
   macro avg       0.63      0.63      0.63    141960
weighted avg       0.63      0.63      0.63    141960

              precision    recall  f1-score   support

           0       0.65      0.62      0.63     70980
           1       0.64      0.66      0.65     70980

   micro avg       0.64      0.64      0.64    141960
   macro avg       0.64      0.64      0.64    141960
weighted avg       0.64      0.64      0.64    141960

              preci

              precision    recall  f1-score   support

           0       0.65      0.62      0.63     70980
           1       0.64      0.66      0.65     70980

   micro avg       0.64      0.64      0.64    141960
   macro avg       0.64      0.64      0.64    141960
weighted avg       0.64      0.64      0.64    141960

              precision    recall  f1-score   support

           0       0.69      0.65      0.67     70980
           1       0.67      0.71      0.69     70980

   micro avg       0.68      0.68      0.68    141960
   macro avg       0.68      0.68      0.68    141960
weighted avg       0.68      0.68      0.68    141960

              precision    recall  f1-score   support

           0       0.74      0.69      0.71     70980
           1       0.71      0.76      0.73     70980

   micro avg       0.72      0.72      0.72    141960
   macro avg       0.72      0.72      0.72    141960
weighted avg       0.72      0.72      0.72    141960

              preci

              precision    recall  f1-score   support

           0       0.92      0.51      0.66     17784
           1       0.14      0.64      0.23      2241

   micro avg       0.53      0.53      0.53     20025
   macro avg       0.53      0.58      0.45     20025
weighted avg       0.83      0.53      0.61     20025

              precision    recall  f1-score   support

           0       0.92      0.54      0.68     17784
           1       0.14      0.61      0.23      2241

   micro avg       0.55      0.55      0.55     20025
   macro avg       0.53      0.57      0.45     20025
weighted avg       0.83      0.55      0.63     20025

              precision    recall  f1-score   support

           0       0.91      0.61      0.73     17784
           1       0.15      0.53      0.23      2241

   micro avg       0.60      0.60      0.60     20025
   macro avg       0.53      0.57      0.48     20025
weighted avg       0.83      0.60      0.68     20025

